In [ ]:
def prepare_sequences(notes, n_vocab):
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))
    print(len(pitchnames))
     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # make compatible input for LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [ ]:
#build the network

def build_lstm(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(256,return_sequences=True,input_shape=(network_input.shape[1], network_input.shape[2])
    ))      
    model.add(LSTM(256,return_sequences=True))
    model.add(LSTM(256))
    model.add(BatchNorm())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics = 'accuracy')

    return model

In [ ]:
def train_network():
    # get the number of different pitch names
    n_vocab = len(set(loaded_data))
    print('n_vocab',n_vocab)
    network_input, network_output = prepare_sequences(loaded_data, n_vocab)

    model = build_lstm(network_input, n_vocab)
    print(model.summary())
    history = train(model, network_input, network_output)
    model.save('/content/drive/My Drive/data/model.h5')
    return history

In [ ]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from google.colab import drive

string_list = ['Bass','Drums','Sax','Piano'] #potencijalni kandidati (bass i piano imaju najvise info)
drive.mount('/content/drive')
with open('/content/drive/My Drive/data/Piano.pickle', 'rb') as handle:
    loaded_data = pickle.load(handle)

history = train_network()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
n_vocab 674
674
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 128)          66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
activation (Activation)      (None, 64)    